# Trakt Integration Smoke Test

This notebook exercises the Warp Media Center Trakt integration end to end.

> **Prerequisites**
>
> * Set the `TRAKT_CLIENT_ID` and `TRAKT_CLIENT_SECRET` environment variables (or configure them via `warp_mediacenter/config/informationproviderservicesettings.json`).
> * Ensure outbound HTTPS access to `api.trakt.tv`.
> * Run the notebook from the project root so relative cache/token paths resolve correctly.


In [ ]:
import time

from warp_mediacenter.backend.information_handlers.models import MediaType
from warp_mediacenter.backend.information_handlers.providers import InformationProviders
from warp_mediacenter.backend.information_handlers.trakt_manager import DeviceAuthPollingError


In [ ]:
providers = InformationProviders(allow_missing_trakt=False)
print(f"Trakt available: {providers.trakt_available()}")
print(f"Existing token: {providers.trakt_has_token()}")


## 1. Start device authentication

Run the cell below to generate a user code and URL. Follow the instructions in the output to approve the application in your browser.


In [ ]:
device = providers.start_trakt_device_auth()
device.model_dump()


## 2. Poll until the device is authorized

This loop handles the `authorization_pending` and `slow_down` responses from Trakt by sleeping for the recommended interval before retrying.


In [ ]:
token = None
while token is None:
    try:
        token = providers.poll_trakt_device_token(device.device_code)
        print("Access token stored on disk.")
    except DeviceAuthPollingError as exc:
        wait_for = exc.retry_interval or device.interval
        if exc.should_retry:
            print(f"Waiting for authorization ({exc.error}); retrying in {wait_for} seconds...")
            time.sleep(wait_for)
        else:
            raise

token.model_dump()


## 3. Manually refresh the token

The `HttpSession` will refresh tokens automatically on `401` responses, but you can trigger a manual refresh to confirm refresh credentials are stored correctly.


In [ ]:
refreshed = providers.refresh_trakt_token()
print(f"Token expires at: {providers.trakt_token_expires_at()}")
refreshed.model_dump()


## 4. Fetch profile and account settings


In [ ]:
profile = providers.get_trakt_profile()
settings = providers.get_trakt_user_settings()

profile.model_dump()


In [ ]:
# Settings payloads include nested dictionaries that vary per account.
# Display the keys to confirm retrieval without dumping sensitive data.
settings_keys = {key: type(value).__name__ for key, value in settings.model_dump().items()}
settings_keys


## 5. Search for a show or movie

Use the Trakt search endpoint to find catalog entries and confirm they are normalised into `CatalogItem` models.


In [ ]:
query = "Inception"
search_results = providers.search_trakt(query, types=[MediaType.MOVIE], limit=5)
[
    {
        "type": result.type.value,
        "score": result.score,
        "title": result.media.title,
        "year": result.media.year,
        "ids": result.media.extra.get("ids"),
    }
    for result in search_results
]
